In [13]:
import cdfsampler
import natjoin
import random

In [33]:
def isPrime(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False

    return True

In [69]:
def normalize(lower, upper, value):
    if lower == upper:
        return .5
    return (float(value) - float(lower)) / (float(upper) - float(lower))

In [99]:
def hash_function(entry_value, prime):
    hash_value = 0
    for element in str(entry_value):
        hash_value += ord(element) % v
    return hash_value

In [190]:
def cdfjoin(tables, sampling_threshold):
    if len(tables) <= 1:
        return tables
    else:
        attrs = {}
        
        #find join attributes
        for i in range(len(tables)):
            table = tables[i]
            for table_key in table[0].keys():
                if table_key in attrs.keys(): attrs[table_key].append(i)
                else: attrs[table_key] = [i]
        join_attrs = {k: v for k,v in attrs.items() if len(v) > 1}
        
        #build dictionary of prime numbers to use when hashing each join attribute
        primes = []
        primes_needed = len(join_attrs.keys())
        lower_bound = 31
        upper_bound = 126
        while len(primes) < primes_needed:
            primes += [i for i in range(upper_bound, lower_bound+upper_bound) if isPrime(i)]
            lower_bound = upper_bound+1
            upper_bound += primes_needed
            
        attrs_hash_dict = {}
        for k, v in join_attrs.items():
            hash_num = random.choice(primes)
            attrs_hash_dict[k] = [hash_num]
            primes.remove(hash_num)
            
        #compute hashes of each key in join attribute dictionary
        for k,v in attrs_hash_dict.items():
            table_indices = join_attrs[k]
            hash_min = hash_function(tables[table_indices[0]][0][k], v)
            hash_max = hash_min
            for index in table_indices:
                table = tables[index]
                for entry in table:
                    hash_score = hash_function(entry[k], v)
                    entry[str(k) + " hash score"] = hash_score
                    if hash_score < hash_min:
                        hash_min = hash_score
                    elif hash_score > hash_max:
                        hash_max = hash_score
            for index in table_indices:
                table = tables[index]
                for entry in table:
                    value = entry[str(k) + " hash score"]
                    entry[str(k) + " hash score"] = normalize(hash_min, hash_max, value)
            attrs_hash_dict[k].append([hash_min, hash_max])

        #compute sums of all hashes i
        for table in tables:
            for entry in table:
                entry["hash sum"] = sum([v for k,v in entry.items() if k[-11:] == " hash score"])
                delete_keys = [k for k,v in entry.items() if k[-11:] == " hash score"]
                for key in delete_keys:
                    del entry[key]
        
        #filter for all entries whose cdf <= sampling probability
        filtered_tables = []
        for i in range(len(tables)):
            table = tables[i]
            filtered_table = []
            if "hash sum" in table[0].keys():
                n_join_attrs = len([i for k,v in join_attrs.items() if i in v])
                for entry in table:
                    if cdfsampler.cdf(n_join_attrs, entry["hash sum"]) <= sampling_threshold:
                        filtered_table.append(entry)
                    del entry["hash sum"]
            if len(filtered_table) > 0:
                filtered_tables.append(filtered_table)

        joined_tables = natjoin.natural_join(filtered_tables)
        return joined_tables
        
            

In [191]:
example_table_one = [{"name": " ", "age": 21, "year": 1957},
                    {"name": "yikes", "age": 21, "year": 1938},
                    {"name": "i'm", "age": 21, "year": 2512},
                    {"name": "so", "age": 21, "year": 1914},
                    {"name": "bleh", "age": 21, "year": 1932},
                    {"name": "right now", "age": 21, "year": 1957}]

example_table_two = [{"name": " ", "age": 21, "g": "a"},
                    {"name": " ", "age": 21, "g": "b"},
                    {"name": " ", "age": 21, "g": "c"},
                    {"name": " ", "age": 42, "g": "d"},
                    {"name": " ", "age": 42, "g": "e"},
                    {"name": " ", "age": 42, "g": "f"}]

example_table_three = [{"name": "nothing good ever happens", "song": "off you"}]
    
print(cdfjoin([example_table_one, example_table_two, example_table_three], .75))

[{'name': ' ', 'age': 21, 'year': 1957, 'g': 'a'}, {'name': ' ', 'age': 21, 'year': 1957, 'g': 'b'}, {'name': ' ', 'age': 21, 'year': 1957, 'g': 'c'}]


In [151]:
print(cdfjoin([example_table_one, example_table_two, example_table_three], .25))

[{'name': ' ', 'age': 21, 'year': 1957, 'g': 'a', 'song': 'off you'}, {'name': ' ', 'age': 21, 'year': 1957, 'g': 'b', 'song': 'off you'}, {'name': ' ', 'age': 21, 'year': 1957, 'g': 'c', 'song': 'off you'}]
